<a href="https://colab.research.google.com/github/Rubnserrano/TF-Certification/blob/main/08_curso_tensorflow_certificate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP first steps (exciting!!!)

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-9895fa8d-bcc5-4974-41ff-436c704d9237)


In [3]:
!wget https://raw.githubusercontent.com/Rubnserrano/TF-Certification/main/extra/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-11-23 12:06:56--  https://raw.githubusercontent.com/Rubnserrano/TF-Certification/main/extra/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5418 (5.3K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   5.29K  --.-KB/s    in 0s      

2022-11-23 12:06:56 (89.9 MB/s) - ‘helper_functions.py’ saved [5418/5418]



In [4]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
unzip_data("nlp_getting_started.zip")

--2022-11-23 12:07:06--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 172.217.194.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-11-23 12:07:06 (97.5 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [5]:
!ls

helper_functions.py	   __pycache__		  test.csv
nlp_getting_started.zip    sample_data		  train.csv
nlp_getting_started.zip.1  sample_submission.csv


In [7]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [8]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
import numpy as np
print(train_df.target.value_counts())

0    4342
1    3271
Name: target, dtype: int64


In [10]:
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [11]:
train_df_shuffled.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [12]:
import random

train_df_shuffled.text[1]

'Forest fire near La Ronge Sask. Canada'

In [13]:
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Sioux City Fire Officials Believe Bridge Collapse Lead To Cement Truck Roll Over - Siouxland Matters: Siouxlan... http://t.co/sZTGmbkoHG

---

Target: 0 (not real disaster)
Text:
That's the ultimate road to destruction

---

Target: 0 (not real disaster)
Text:
BESTSELLER! Fossil Dawson Mini Cross Body Bag EsÛ_ $98.00 http://t.co/HhHvKxFAIH

---

Target: 1 (real disaster)
Text:
Here's what may have caused a Metro train to derail in downtown D.C. http://t.co/mEiSNKv5Tb

---

Target: 0 (not real disaster)
Text:
i'd still be team usagi even if she blew up the entire solar system by some airhead misstep

---



In [14]:
from sklearn.model_selection import train_test_split

In [16]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state=42)

In [18]:
print(f"Train sentences: {len(train_sentences)} and train labels: {len(train_labels)}")
print(f"Train sentences: {len(val_sentences)} and train labels: {len(val_labels)}")

Train sentences: 6851 and train labels: 6851
Train sentences: 762 and train labels: 762


In [20]:
train_sentences[:5], train_labels[:5]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
       dtype=object), array([0, 0, 1, 0, 0]))